Este segundo notebook es para entrenar un modelo de aprendizaje no supervisado utiliznado el dataset 'test.arquet' el cual nos ayudará a agrupar los datos en 3 categorías: 'low', 'medium', 'hight'.

In [180]:
#importamos las librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [181]:

test_df=pd.read_parquet(r"Datasets/test.parquet")


In [182]:
test_df.isnull().sum()

id                             0
url                            0
region                         0
region_url                     0
type                           0
sqfeet                         0
beds                           0
baths                          0
cats_allowed                   0
dogs_allowed                   0
smoking_allowed                0
wheelchair_access              0
electric_vehicle_charge        0
comes_furnished                0
laundry_options             7855
parking_options            14005
image_url                      0
description                    0
lat                          196
long                         196
state                          0
dtype: int64

borramos los caracteristicas nulas

In [183]:
test_df.dropna(axis=1,inplace=True)

In [184]:
test_df.isnull().sum()

id                         0
url                        0
region                     0
region_url                 0
type                       0
sqfeet                     0
beds                       0
baths                      0
cats_allowed               0
dogs_allowed               0
smoking_allowed            0
wheelchair_access          0
electric_vehicle_charge    0
comes_furnished            0
image_url                  0
description                0
state                      0
dtype: int64

In [185]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

test_df['region']=labelencoder.fit_transform(test_df['region'])
test_df['type']=labelencoder.fit_transform(test_df['type'])
test_df['baths']=labelencoder.fit_transform(test_df['baths'])
test_df['state']=labelencoder.fit_transform(test_df['state'])

test_df.head(3)

,id,url,region,region_url,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,image_url,description,state
0,7037609789,https://annarbor.craigslist.org/apa/d/wixom-ho...,9,https://annarbor.craigslist.org,8,1344,3,3,0,0,1,0,0,0,https://images.craigslist.org/00M0M_iNczP1nzIL...,"OPEN HOUSE TODAY! APPLY THIS WEEK, PUT A HOLDI...",22
1,7032406876,https://vermont.craigslist.org/apa/d/randolph-...,370,https://vermont.craigslist.org,0,1050,2,1,0,0,1,0,0,0,https://images.craigslist.org/00L0L_ecirmYBIzL...,"Think of it, you'll be first to get your mail....",46
2,7037022682,https://annarbor.craigslist.org/apa/d/ann-arbo...,9,https://annarbor.craigslist.org,0,1150,2,3,1,1,1,1,0,0,https://images.craigslist.org/00e0e_dPln2xjo9g...,One of Ann Arbor's most luxurious apartment co...,22


borramos las columnas que contienen texto

In [186]:
test_df.drop(['id','url','region_url','image_url', 'description'], axis=1, inplace=True)

In [187]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38498 entries, 0 to 38497
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   region                   38498 non-null  int32
 1   type                     38498 non-null  int32
 2   sqfeet                   38498 non-null  int64
 3   beds                     38498 non-null  int64
 4   baths                    38498 non-null  int64
 5   cats_allowed             38498 non-null  int64
 6   dogs_allowed             38498 non-null  int64
 7   smoking_allowed          38498 non-null  int64
 8   wheelchair_access        38498 non-null  int64
 9   electric_vehicle_charge  38498 non-null  int64
 10  comes_furnished          38498 non-null  int64
 11  state                    38498 non-null  int32
dtypes: int32(3), int64(9)
memory usage: 3.1 MB


In [188]:
from sklearn.cluster import KMeans
X = test_df[['region','type','sqfeet', 'beds','baths','cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',  'state']]


In [189]:
#instanciamos Kmeans
model = KMeans(n_clusters=3, random_state=42) 

#entremos el modelo
test_kmeans = model.fit(X)

In [190]:
#aqui, corroboramos la etiqueta que el mododelo asocio a cada una de
#las instancias.
labels = test_kmeans.labels_
centroid = test_kmeans.cluster_centers_

In [191]:
from sklearn.metrics import silhouette_score

# X es el conjunto de datos
# labels es el arreglo de etiquetas asignadas para el modelo k-means
score = silhouette_score(X, labels, metric='euclidean')
print("Puntaje de silueta: {:.3f}".format(score))


Puntaje de silueta: 0.563


escogiendo las tres mejores caracteristicas

In [192]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# Selección de las mejores 3 características
selector = SelectKBest(score_func=mutual_info_classif, k=3)
X_new = selector.fit_transform(X, labels)

# Entrenamiento del modelo k-means con las características seleccionadas
model = KMeans(n_clusters=3, random_state=42) 
test_kmeans = model.fit(X_new)

# Evaluación del modelo con la métrica Silhouette Score
labels = test_kmeans.labels_
score = silhouette_score(X_new, labels, metric='euclidean')
print("Puntaje de silueta: {:.3f}".format(score))


Puntaje de silueta: 0.624


In [193]:
len(labels)

38498

In [194]:
clusters=pd.DataFrame(labels, columns=['pred'])
clusters.to_csv('CamiiloP14(1).csv', index=False)

In [195]:
pd.read_csv('CamiiloP14(1).csv')

,pred
0,0
1,1
2,1
3,0
4,1
...,...
38493,1
38494,1
38495,1
38496,0
